## process a synthetic chorale manufactured by TonicNet
TonicNet can produce a synthetic chorale using the main.py module:

In [1]:
!ls -lth ./eval | head 
print(f'Hello World')

total 5.1M
drwxr-xr-x. 1 prent prent 139K Mar  8 11:03 numpy_chorales
drwxr-xr-x. 1 prent prent 130K Mar  8 11:03 midi_samples
-rw-r--r--. 1 prent prent  158 Mar  7 15:26 sample_smoothed.mid
drwxr-xr-x. 1 prent prent  246 Mar  5 10:18 __pycache__
drwxr-xr-x. 1 prent prent   66 Jan 25 09:04 samples
-rw-r--r--. 1 prent prent 205K Jul 15  2022 TonicNet_epoch-58_loss-0.317_acc-90.928.pt
-rw-r--r--. 1 prent prent 4.9M Jul 14  2022 TonicNet_epoch-58_loss-0.322_acc-90.745.pt
-rw-r--r--. 1 prent prent 7.7K Jul 13  2022 sample.py
-rw-r--r--. 1 prent prent 3.1K Jul 12  2022 eval.py
Hello World


In [2]:
import numpy as np
import copy
import mido
import time
from importlib import reload
import fluidsynth
from IPython.display import Audio, display
import music21 as m
import os
import muspy
import pandas as pd
import sys
sys.path.insert(0, '/home/prent/Dropbox/Tutorials/coconet-pytorch/coconet-pytorch-csound')
# import piano as p
# import selective_stretching_codes as stretch
import samples_used as su
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
soundfont = '../coconet-pytorch/font.sf2' # you will need to download this from location specified in the github README.md
midi_dir = 'eval/midi_samples'
numpy_dir = 'eval/numpy_chorales'
CSD_FILE = 'string_orc.csd'
LOGNAME = 'string_orc.log'
WAV_OUT = '/home/prent/Music/sflib/string_orc.wav'
model_path = 'eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt'
downbeat = 1 # all the synthetic chorales out of TonicNet have a downbeat of 1
keys = ['C','C#','D','D#','E','F','F#','G','G#','A','B-','B']

In [3]:
!pwd
!ls -lth eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt
!python main.py -s
# the "smoothed" algorithm doesn't work.

/home/prent/Dropbox/Tutorials/TonicNet
-rw-r--r--. 1 prent prent 4.9M Jul 14  2022 eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt
loded params from eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt
TonicNet(
  (embedding): Embedding(98, 256)
  (pos_emb): Embedding(64, 0)
  (z_embedding): Embedding(80, 32)
  (dropout_i): VariationalDropout()
  (rnn): GRU(288, 256, num_layers=3, batch_first=True)
  (dropout_o): VariationalDropout()
  (hidden_to_tag): Linear(in_features=288, out_features=98, bias=False)
)

0
	 0 : (10, 'major')
ending
SAVED sample to ./eval/sample.mid
/home/prent/Dropbox/Tutorials/TonicNet/eval/utils.py:120: StreamIteratorInefficientWarning: flat is not defined on StreamIterators. Call .stream() first for efficiency
  for n in part.notesAndRests.flat:
SAVED rhythmically 'smoothed' sample to ./eval/sample_smoothed.mid


## Use the TonicNet sample generator to create and save synthetic chorales as numpy files
Steps:
- generate a sythetic chorale as a music21 stream.
- convert that to the chorale format, (notes, voices) by time-interval in piano_roll format. 

In [4]:
from eval.sample import sample_TonicNet_random
from eval.utils import plot_loss_acc_curves, indices_to_stream, smooth_rhythm

In [5]:
def midi_to_input(midi_file):
    music = muspy.read(midi_file)
    if music.key_signatures != []: # check if the midi file includes a key signature - some don't
        root = music.key_signatures[0].root 
        mode = music.key_signatures[0].mode # major or minor
    else: 
        print('Warning: no key signature found. Assuming C major')
        mode = "major"
        root = 0    
    if music.time_signatures != []: # check if the midi file includes a time signature - some don't
        numerator = music.time_signatures[0].numerator
        denominator = music.time_signatures[0].denominator 
    else: 
        print('Warning: no time signature found. Assuming 4/4')
        numerator = 4
        denominator = 4
    # turn it into a piano roll
    piano_roll = muspy.to_pianoroll_representation(music,encode_velocity=False) # boolean piano roll if False, default True
    # print(piano_roll.shape) # should be one time step for every click in the midi file
    q = music.resolution # quarter note value in this midi file. 
    q16 = q // 4 # my desired resolution is by 1/16th notes
    print(f'time signatures: {numerator}/{denominator}')
    time_steps = piano_roll.shape[0] // q16
    print(f'music.resolution is q: {q}. q16: {q16} time_steps: {time_steps} 1/16th notes')
    sample= np.zeros(shape=(time_steps,4)).astype(int) # default is float unless .astype(int)
    # This loop is able to load an array of shape N,4 with the notes that are being played in each time step
    for click in range(0,piano_roll.shape[0],q16): # q16 is skip 240 steps for 1/16th note resolution
        voice = 3 # start with the low voices and decrement for the higher voices as notes get higher
        for i in range(piano_roll.shape[1]): # check if any notes are non-zero
              time_interval = (click) // q16 
              if (piano_roll[click][i]): # if velocity anything but zero - unless you set encode_velocity = False
              # if time_interval % 16 == 0:
              #     print(f'time step: {click} at index {i}, time_interval: {time_interval}, voice: {voice}')
              # i is the midi note number. I want to transpose it into C
                    sample[time_interval][voice] = i - root # index to the piano roll with a note - transposed by the key if not C which is 0
                    voice -= 1 # next instrument will get the higher note
    return (sample,root,mode)           

## Manufacture synthetic chorales in bulk

- Call the TonicNet synthesize function <code>sample_TonicNet_random(load_path = model_path, temperature=1.0)</code>
- This creates a MIDI file called <code>sample.mid</code> in the <code>eval</code> directory
- Load that MIDI file into a chorale structure of (voices, time_steps)
- Save that chorale as a numpy file
- Rename the <code>sample.mid</code> in the <code>eval</code> directory as <code>sample#.mid</code> where # is a number of the chorale in the range
- Read the stream into a Music21 data structure so that we can determine the key and the mode of the chorale
- At the end of the process, the <code>eval</code> directory has all the MIDI synthetic chorales, and the <code>eval/numpy_chorales</code> as all the numpy arrays of chorales in (voices, time_step) format

In [6]:
sample_name = os.path.join('eval','sample.mid')
print(f'{midi_dir =}, {numpy_dir = }, {sample_name = }')

midi_dir ='eval/midi_samples', numpy_dir = 'eval/numpy_chorales', sample_name = 'eval/sample.mid'


In [7]:
# Synthesize a number of chorales as MIDI files and numpy files
for synth_chorale in range(5000, 5001): # 
    # generate a synthetic chorale as a tensor
    x = sample_TonicNet_random(load_path = model_path, temperature=1.0)
    # create a sample.mid file in the eval directory in the same directory as the model_path
    indices_to_stream(x, return_stream = False)
    print(f'about to read in {sample_name = }')
    sample, _, _ = midi_to_input(sample_name) # sample its notes, voices
    chorale = sample.transpose()
    numpy_file = os.path.join(numpy_dir, 'chorale' + str(synth_chorale))
    np.save(numpy_file, chorale)
    output_name = os.path.join(midi_dir, 'sample' + str(synth_chorale) + '.mid')
    print(f'moving {sample_name = } to {output_name = }')
    os.rename(sample_name, output_name)
    m21_stream = indices_to_stream(x, return_stream = True) # this converts the tensor to a music21 stream for analysis
    fis = str(m21_stream.analyze('key'))
    print(f'music21 says: {fis = }')
    key_name, mode = fis.split()
    print(f'after split. {key_name = }, {mode = }')
    i = 0
    for key in keys:
        if key.upper() == key_name.upper():
            root = i
            print(f'found a match between music21 {key_name = }, and {key = } as {root = }, {mode = }')
        i += 1    
    

loded params from eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt
TonicNet(
  (embedding): Embedding(98, 256)
  (pos_emb): Embedding(64, 0)
  (z_embedding): Embedding(80, 32)
  (dropout_i): VariationalDropout()
  (rnn): GRU(288, 256, num_layers=3, batch_first=True)
  (dropout_o): VariationalDropout()
  (hidden_to_tag): Linear(in_features=288, out_features=98, bias=False)
)

0
	 0 : (3, 'major')
ending
SAVED sample to ./eval/sample.mid
about to read in sample_name = 'eval/sample.mid'
time signatures: 4/4
music.resolution is q: 1024. q16: 256 time_steps: 263 1/16th notes
moving sample_name = 'eval/sample.mid' to output_name = 'eval/midi_samples/sample5000.mid'
music21 says: fis = 'C major'
after split. key_name = 'C', mode = 'major'
found a match between music21 key_name = 'C', and key = 'C' as root = 0, mode = 'major'


In [8]:
synth_chorale = 4888
file_name = 'eval/midi_samples/sample' + str(synth_chorale) + '.mid'
sample, root, mode = midi_to_input(file_name) # sample is notes, voices by time interval
print(*sample[:10],sep='\n')

time signatures: 4/4
music.resolution is q: 1024. q16: 256 time_steps: 504 1/16th notes
[68 63 60 53]
[68 63 60 53]
[68 63 60 53]
[68 63 60 53]
[70 63 58 55]
[70 63 58 55]
[70 63 58 55]
[70 63 58 55]
[ 0 71 63 56]
[ 0 71 63 56]
